In [1]:
# Dataset

import os
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms


class UCSDPed2Dataset(Dataset):
    def __init__(self, root_dir, sequence_length=5, image_size=128, mode="train"):
        self.sequence_length = sequence_length
        self.image_size = image_size
        self.mode = mode

        if mode == "train":
            self.video_dir = os.path.join(root_dir, "train")
        else:
            self.video_dir = os.path.join(root_dir, "test")

        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor()
        ])

        self.samples = []
        self._prepare_samples()

    def _prepare_samples(self):
        video_folders = sorted(os.listdir(self.video_dir))

        for video in video_folders:
            video_path = os.path.join(self.video_dir, video)

            if not os.path.isdir(video_path):
                continue

            frames = sorted([
                f for f in os.listdir(video_path)
                if f.lower().endswith((".jpg", ".png", ".tif"))
            ])

            for i in range(len(frames) - self.sequence_length):
                self.samples.append((
                    video_path,
                    frames[i:i + self.sequence_length],
                    frames[i + self.sequence_length]
                ))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_path, input_frames, target_frame = self.samples[idx]

        input_tensor = []
        for f in input_frames:
            img = Image.open(os.path.join(video_path, f)).convert("L")
            img = self.transform(img)
            input_tensor.append(img)

        input_tensor = torch.stack(input_tensor, dim=0)

        target_img = Image.open(
            os.path.join(video_path, target_frame)
        ).convert("L")
        target_img = self.transform(target_img)

        return input_tensor, target_img


In [2]:
# Model

import torch.nn as nn


class CNNEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, 2, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, 2, 1),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, 2, 1),
            nn.ReLU()
        )

    def forward(self, x):
        return self.encoder(x)


class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.conv = nn.Conv2d(
            input_dim + hidden_dim,
            4 * hidden_dim,
            3,
            padding=1
        )

    def forward(self, x, h, c):
        combined = torch.cat([x, h], dim=1)
        i, f, o, g = torch.split(self.conv(combined), self.hidden_dim, dim=1)

        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        o = torch.sigmoid(o)
        g = torch.tanh(g)

        c = f * c + i * g
        h = o * torch.tanh(c)
        return h, c


class ConvLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.cell = ConvLSTMCell(input_dim, hidden_dim)

    def forward(self, x):
        B, T, C, H, W = x.size()
        h = torch.zeros(B, C, H, W, device=x.device)
        c = torch.zeros_like(h)

        for t in range(T):
            h, c = self.cell(x[:, t], h, c)

        return h


class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 4, 2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(x)


class FutureFramePredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = CNNEncoder()
        self.convlstm = ConvLSTM(128, 128)
        self.decoder = Decoder()

    def forward(self, x):
        encoded = torch.stack(
            [self.encoder(x[:, t]) for t in range(x.size(1))],
            dim=1
        )
        h = self.convlstm(encoded)
        return self.decoder(h)


In [3]:
# Training

from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = FutureFramePredictor().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

train_dataset = UCSDPed2Dataset(
    root_dir="../datasets/UCSDped2",
    sequence_length=5,
    image_size=128,
    mode="train"
)

train_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=0
)

num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, target in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs = inputs.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        pred = model(inputs)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} Avg Loss: {running_loss / len(train_loader):.6f}")

# ✅ SAVE MODEL
torch.save(model.state_dict(), "baseline_ucsd_ped2.pth")


Epoch 1/50:   2%|█▏                                                                   | 11/618 [00:10<09:23,  1.08it/s]


KeyboardInterrupt: 

In [ ]:
# testing

test_dataset = UCSDPed2Dataset(
    root_dir="../datasets/UCSDped2",
    sequence_length=5,
    image_size=128,
    mode="test"
)

test_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False
)

model.load_state_dict(torch.load("baseline_ucsd_ped2.pth", map_location=device))
model.eval()

errors = []

with torch.no_grad():
    for inputs, target in tqdm(test_loader, desc="Testing"):
        inputs = inputs.to(device)
        target = target.to(device)

        pred = model(inputs)
        mse = torch.mean((pred - target) ** 2).item()
        errors.append(mse)


In [ ]:
# visualization

import matplotlib.pyplot as plt

# Error curve
plt.figure(figsize=(12,4))
plt.plot(errors)
plt.title("Prediction Error on UCSD Ped2 Test Set")
plt.xlabel("Frame Index")
plt.ylabel("MSE")
plt.grid()
plt.show()

# Frame visualization
with torch.no_grad():
    inputs, target = test_dataset[200]
    inputs = inputs.unsqueeze(0).to(device)
    target = target.unsqueeze(0).to(device)

    pred = model(inputs)

    plt.figure(figsize=(8,4))
    plt.subplot(1,2,1)
    plt.title("Ground Truth")
    plt.imshow(target[0,0].cpu(), cmap="gray")
    plt.axis("off")

    plt.subplot(1,2,2)
    plt.title("Predicted")
    plt.imshow(pred[0,0].cpu(), cmap="gray")
    plt.axis("off")
    plt.show()
